In [4]:
import re
import json
import requests
import urllib.request
import pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
import numpy as np


In [43]:
url = 'https://www.banxico.org.mx/ServiciosFinancieros/sanciones/publicacionesInter.xhtml?idTipoDisposicion=2&idDisposicion=0'

In [44]:
html= requests.get(url).content

In [45]:

soup=BeautifulSoup(html,'html')

In [46]:
#Para extraer de la catergoria "td a" la informacion a los intermediarios y disposicion
links = soup.select('td a')

In [47]:
#para extraer la informacion parcial de los links sobre los intermediarios y disposicion, que posteriormente se usara, usamos rgex para extraer lo siguienteej. 'Disposicion=2&idIntermediario=37&'
base_inte_disp = []


base = [links[i]['href'] for i in range(len(links))]

for b in base:
    clean = re.findall('Disposicion.*?&.*?&', b)
    base_inte_disp.append(clean)
random = np.random.choice

time.sleep(random(30))

In [48]:
#para hacer una lista con los links paciales, ya que esta como lista de listas
base_inte_disp_clean = [clean for ba in base_inte_disp for clean in ba]

In [49]:
#vamos iterando cada intermediario-disposcion en cada anio, asi como por pecunaria o no pecunaria, en cada request de anadio un tiempo maximo de esper de 60segundos
#se extrae la tabla de cada pagina web que esta bajo el id sanciones:sanciones y lo convertimos en un datafarme
df=pd.DataFrame()
year= [2016,2017,2018,2019,2020]
pecunaria = ['false','true']
for bid in base_inte_disp_clean:
    for y in year:
        for p in pecunaria:
            time.sleep(random(60))
            url = f'https://www.banxico.org.mx/ServiciosFinancieros/sanciones/publicaciones.xhtml?id{bid}anio={y}&pecuniaria={p}&tipoConsulta=I'
            html= requests.get(url).content
            soup=BeautifulSoup(html,'html')
            tbl = soup.find("table",{"id":"sanciones:sanciones"})
            temp = pd.read_html(str(tbl))[0]
            df = df.append(temp)

In [50]:
df

,Denominación social,Precepto legal infringido (vigente en la fecha en que se cometió la infracción),Incumplimiento,Sanción,Medio de defensa,Vigencia,Aclaraciones,Fecha de Cobro
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"Casa de Bolsa Banorte Ixe, S.A. de C.V., Grupo...","Numeral CB.2, tercer párrafo, inciso d), de la...",Falta a las disposiciones de operaciones de Re...,"Monto $4,688,314.40 Fecha de imposición03-05-2016",Ninguno,NaN,Esta conducta se considera graveNo es reincide...,24-05-2016
1,NaN,NaN,NaN,"$4,688,314.40",NaN,NaN,NaN,NaN
2,"Value, S.A. de C.V., Casa de Bolsa, Value Grup...","Numeral 6.1, de las Reglas de Fideicomiso.",Falta a las Reglas para las operaciones de fid...,"Monto $14,956.00 Fecha de imposición10-11-2016",Ninguno,NaN,Esta conducta no se considera graveNo es reinc...,01-12-2016
3,NaN,NaN,NaN,"$14,956.00",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"Círculo de Crédito, S.A. de C.V., Sociedad de ...","Numerales 5.1 y 5.2 del considerando Quinto, i...",Falta a las reglas generales a las que deberán...,"Monto $35,067.02 Fecha de imposición28-07-2017",Amparo,NaN,Esta conducta no se considera graveNo es reinc...,26-07-2019
1,NaN,NaN,NaN,"$35,067.02",NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
data = df.to_csv('banxico.csv', index=False)

# Data Cleaning

In [7]:
#fucion para clasificion de reincidencia de las sanciones
def clasificar_reincidente(x):
    if re.findall('No es reincidente',x):
        return "No"
    else:
        return "Si"
    
#fucion para clasificion si son graves o no la multas 
def clasificar_grave(x):
    if re.findall('no se considera grave',x):
        return "No"
    else:
        return "Si"
#funcion para cambiar el formato a fecha
def cambio_fecha(x):
    return pd.to_datetime(x)    


def formato(path):
    #extraccion de informacion
    data= pd.read_csv(path)
    
    #cambio de nombre de las columnas
    data.columns = ['nombre', 'precepto', 'incumplimiento', 'sancion', 'defensa', 'vigencia', 'aclaraciones', 'fecha_cobro']
    
    #quitar las filas que no tienen datos en la columna de nombre
    data=data.drop(data[data['nombre'].isnull()==True].index, axis=0)
    
    #separacion de la informacion del monto de la sancion y los espacios vacios llenarlos con 0 
    data['sancion'] = data['sancion'].astype('str') 
    data ["monto"] = data ["sancion"].str.extract(r'([\d].*\.\d.)')
    data ["monto"] = data ["monto"].str.replace(',', '')
    data [['monto']] = data[['monto']].fillna(0).astype(float)
    
    #separacion de la informacion de la fecha de la sancion 
    data ["fecha_imposicion"] = data ["sancion"].str.extract(r'(\d.-\d.-\d...)')
    
    #eliminacion de la columna sancion
    data=data.drop('sancion', axis=1)
    
    #cambio al formato de fecha para las columnas que son fecha
    data[['fecha_cobro','fecha_imposicion','vigencia']] = data[['fecha_cobro','fecha_imposicion','vigencia']].apply(cambio_fecha)
    
    #limpieza de los nombres de la entidades retirando extension legal
    data ["nombre"] = data ["nombre"].str.extract(r'(\w.*(?=, S))')
    data ["nombre"] = data ["nombre"].str.extract(r'((\w.*(?=, S))|\w.*)')
    
    #extraer la informacion de la circular que se esta infringiendo
    data ["circular"] = data ["precepto"].str.extract(r'(\d*\/\d.*[^.])')
    
    #extraer y clasificar si hay reincidencia
    data['reincidente'] = data['aclaraciones'].apply(clasificar_reincidente)
    data[['reincidente_no', 'reincidente_si']] = pd.get_dummies(data['reincidente'])
    
    #extraer y clasificar si es grave
    data['grave'] = data['aclaraciones'].apply(clasificar_grave)
    data[['grave_no', 'grave_si']]= pd.get_dummies(data['grave'])
    
    #cambio de de fenfesa a categoria
    data['defensa']= data['defensa'].astype('category')
    
    #creacion de nuevo archivo de excel
    data=data.reset_index(drop=True)
    data.to_excel('Banxico_clean.xlsx', index=False)
    
    return 'Archivo guardado'

In [8]:
formato('banxico.csv')

'Archivo guardado'